:toc

## Install dependencies

In [ ]:
import os
import pathlib
import sys

!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
conda_prefix_path = pathlib.Path('conda-env')
site_package_path = conda_prefix_path / 'lib/python3.7/site-packages'
sys.path.append(str(site_package_path.resolve()))
CONDA_PREFIX = str(conda_prefix_path.resolve())
PATH = os.environ['PATH']
LD_LIBRARY_PATH = os.environ.get('LD_LIBRARY_PATH', '')
%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}
%env LD_LIBRARY_PATH={CONDA_PREFIX}/lib:{LD_LIBRARY_PATH}
!bin/micromamba create --yes --prefix $CONDA_PREFIX
!echo 'python ==3.7*' >> {CONDA_PREFIX}/conda-meta/pinned
!bin/micromamba install --yes --prefix $CONDA_PREFIX \
                        --channel litex-hub \
                        --channel main \
                        open_pdks.sky130a \
                        magic \
                        openroad \
                        netgen \
                        yosys \
                        verilator
!bin/micromamba install --yes --prefix $CONDA_PREFIX \
                        --channel conda-forge \
                        tcllib gdstk pyyaml click

## make directories

In [2]:
%mkdir rtl
%mkdir data
%mkdir tb
%mkdir util

## Write Verilog

- Vthernet.v : top module
- Vthernet_RX_MAC.v : Receive MAC
- Vthernet_TX_MAC.v : Transmission MAC


### Vthernet.v

In [3]:
%%writefile rtl/Vthernet.v
`default_nettype none
module Vthernet (
    input   wire        rst,
    input   wire        clk,

    // GMII interface
    output  reg         GTX_CLK,
    output  reg         TX_EN,
    output  reg [7:0]   TXD,
    output  reg         TX_ER,

    input   wire        RX_CLK,
    input   wire        RX_DV,
    input   wire [7:0]  RXD,
    input   wire        RX_ER,

    output  reg         MDC,
    inout   reg         MDIO
);

    // Wishbone interface
    wire        wb_clk_i;
    wire        wb_rst_i;
    wire        wbs_stb_i;
    wire        wbs_cyc_i;
    wire        wbs_we_i;
    wire  [3:0] wbs_sel_i;
    wire [31:0] wbs_dat_i;
    wire [31:0] wbs_adr_i;
    wire        wbs_ack_o;
    wire [31:0] wbs_dat_o;
    
    Vthernet_RX_MAC Vthernet_RX_MAC(
        .rst(rst),
        .clk(clk),
        
        // CSRs
        .my_mac_addr(),
        .ethernet_len_type(),
        .dst_mac_addr(),
        .src_mac_addr(),
        
        // CPU interface
        .rx_irq(),
        
        // GMII interface
        .RX_CLK(RX_CLK),
        .RX_DV(RX_DV),
        .RXD(RXD),
        .RX_ER(RX_ER),
        
        // Memory interface
        .rx_mem_wen(),
        .rx_mem_data(),
        .rx_mem_addr()
    );
    
    Vthernet_TX_MAC Vthernet_TX_MAC(
        // CSRs
        // Write Only
        .my_mac_addr(),
        // Read Only
        // GMII interface
        .GTX_CLK(GTX_CLK),
        .TX_EN(TX_EN),
        .TXD(TXD),
        .TX_ER(TX_ER),
        // Memory interface
        .tx_mem_valid(),
        .tx_mem_data()
    );
endmodule
`default_nettype wire

Writing rtl/Vthernet.v


### Vthernet_RX_MAC.v

In [4]:
%%writefile rtl/Vthernet_RX_MAC.v
//
//  ethernet --- IP ---- UDP
//            |       |
//            -- ARP  -- TCP
//

`default_nettype none
module Vthernet_RX_MAC (
    input   wire        rst,
    input   wire        clk,
    
    // CSRs
    input wire [47:0] 

    // GMII interface
    input   wire        RX_CLK,
    input   wire        RX_DV,
    input   wire [7:0]  RXD,
    input   wire        RX_ER,

    // CPU interface
    output  wire        rx_irq,

    // Memory interface
    output  wire        rx_mem_wen,
    output  wire [7:0]  rx_mem_data,
    output  reg [31:0]  rx_mem_addr
);
    parameter OCT   = 8;
    parameter PRE   = 8'b10101010;
    parameter SFD   = 8'b10101011;
    parameter IPV4  = 16'h0800;

    // Vthernet CSR
    wire [OCT*4-1:0] offload_csr;
    
    wire [OCT*6-1:0] mac_addr;

    wire [OCT*6-1:0] rx_src_mac;

    wire [OCT*2-1:0] rx_ethernet_len_type;

    // RX Memory logic

    always @(posedge RX_CLK) begin
        if(rst) begin
            rx_addr <= 32'h0000_0000;
        end else begin
            if(rx_data_v) begin
                rx_addr <= rx_addr + 32'h0000_0001;
            end else begin
                rx_addr <= 32'h0000_0000;
            end
        end
    end

    // SMI logic
    // transmit logic
    // receive logic
    wire                rx_ethernet_data_v;
    wire    [OCT-1:0]   rx_ethernet_data;
    wire                rx_ipv4_data_v;
    wire    [OCT-1:0]   rx_ipv4_data;
    wire                rx_udp_data_v;
    wire    [OCT-1:0]   rx_udp_data;

    // receive irq signal
    wire                rx_ethernet_irq;
    wire                rx_ipv4_irq;
    wire                rx_udp_irq;
    assign rx_irq = (&offload_csr[1:0]) ? rx_udp_irq : 
                      offload_csr[0]    ? rx_ipv4_irq 
                                        : rx_ethernet_irq;
    assign rx_data_v = (&offload_csr[1:0]) ? rx_udp_data_v :
                         offload_csr[0]    ? rx_ipv4_data_v
                                           : rx_ethernet_data_v;
    assign rx_data  = (&offload_csr[1:0]) ? rx_udp_data :
                         offload_csr[0]   ? rx_ipv4_data
                                          : rx_ethernet_data;

    rx_ethernet #(
        .OCT    (OCT    ),
        .PRE    (PRE    ),
        .SFD    (SFD    ),
        .IPV4   (IPV4   )
    ) rx_ethernet_inst(
        .rst            (rst        ),
        .mac_addr       (mac_addr   ),
        .rx_ethernet_irq(rx_ethernet_irq   ),
        .rx_src_mac     (rx_src_mac ),
        .rx_len_type    (rx_ethernet_len_type   ),
        .RX_CLK         (RX_CLK     ),
        .RX_DV          (RX_DV      ),
        .RXD            (RXD        ),
        .RX_ER          (RX_ER      ),
        .rx_ethernet_data_v (rx_ethernet_data_v ),
        .rx_ethernet_data   (rx_ethernet_data   )
    );

endmodule
`default_nettype wire

Overwriting rtl/Vthernet_RX_MAC.v


## Write Testbench

## Install OpenLANE

In [3]:
!git clone --depth=1 https://github.com/The-OpenROAD-Project/OpenLane

Cloning into 'OpenLane'...
remote: Enumerating objects: 502, done.
remote: Counting objects: 100% (502/502), done.
remote: Compressing objects: 100% (421/421), done.
remote: Total 502 (delta 75), reused 284 (delta 33), pack-reused 0
Receiving objects: 100% (502/502), 3.34 MiB | 6.07 MiB/s, done.
Resolving deltas: 100% (75/75), done.
